In [1]:
%pip install -q pymupdf PyPDF2 transformers accelerate

Note: you may need to restart the kernel to use updated packages.


In [2]:
from huggingface_hub import login
login()

In [3]:
%pip install -q mistral_inference langchain

Note: you may need to restart the kernel to use updated packages.


In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch


model_name = "gemini-1.5-flash-latest" 

# Load the model and tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.float16, device_map="auto")


OSError: gemini-1.5-flash-latest is not a local folder and is not a valid model identifier listed on 'https://huggingface.co/models'
If this is a private repository, make sure to pass a token having permission to this repo either by logging in with `huggingface-cli login` or by passing `token=<your_token>`

In [ ]:
def generate_text(prompt, max_length=100, num_return_sequences=1):
    inputs = tokenizer(prompt, return_tensors="pt")
    outputs = model.generate(
        **inputs,
        max_length=max_length,
        num_return_sequences=num_return_sequences,
        do_sample=True,
        top_k=50,
        top_p=0.95,
        temperature=0.7,
    )
    return [tokenizer.decode(output, skip_special_tokens=True) for output in outputs]

In [ ]:
from langchain.output_parsers import ResponseSchema, StructuredOutputParser
from langchain.prompts import PromptTemplate
import PyPDF2
import re

# 1. Explain the research in EASY Sentences

## 1. SECTION EXTRACTION

In [ ]:
SECTION_PATTERNS = {
    "abstract": r'\babstract\b',
    "introduction": r'\bintroduction\b',
    "methodology": r'\b(methodology|methods|materials and methods)\b',
    "results": r'\b(results|findings)\b',
    "conclusion": r'\b(conclusion|conclusions|discussion and conclusion)\b'
}
SECTION_REGEX = {k: re.compile(v, re.IGNORECASE) for k, v in SECTION_PATTERNS.items()}

def extract_text_from_pdf(pdf_path):
    with open(pdf_path, 'rb') as file:
        reader = PyPDF2.PdfReader(file)
        text = ""
        for page in reader.pages:
            text += page.extract_text() + "\n"
        return text

def extract_sections(text):
    text = re.sub(r'\s+', ' ', text).strip()
    combined_pattern = r'|'.join(f'(?P<{k}>{v})' for k, v in SECTION_PATTERNS.items())
    section_header_regex = re.compile(combined_pattern, re.IGNORECASE)

    sections = {}
    matches = list(section_header_regex.finditer(text))

    for i, match in enumerate(matches):
        section_name = match.lastgroup
        start = match.start()
        end = matches[i + 1].start() if i + 1 < len(matches) else len(text)
        content = text[start:end]
        sections[section_name] = content.strip()

    return sections

## 2. EXPLANATION FUNCTION

In [ ]:
def extract_json_block(text):
    pattern = r'```json\s*(.*?)\s*```'
    matches = re.findall(pattern, text, re.DOTALL)

    return f"```json\n{matches[-1]}\n```"

In [ ]:
def Simplify_research_part(content):
    explanation_schema = ResponseSchema(
        name="explanation",
        description="A simplified and clear explanation of the provided research text."
    )
    output_parser = StructuredOutputParser.from_response_schemas([explanation_schema])
    format_instructions = output_parser.get_format_instructions()

    prompt_template = """\
    You are a helpful assistant that explains research papers in a simple way.

    TASK:
    - Read the provided research content carefully.

    - Return a clear and simplified explanation in one paragraph using plain, accurate language.

    - Avoid adding any information that is not in the original content.

    - The goal is to help anyone understand this part, even without a research background, while staying true to the original meaning.

    Content:
    {content}

    {format_instructions}
    """
    prompt = PromptTemplate(
        template=prompt_template,
        input_variables=["content", "format_instructions"]
    )

    formatted_prompt = prompt.format(content=content, format_instructions=format_instructions)

    response = generate_text(formatted_prompt, max_length=2000, num_return_sequences=1)
    final_response = extract_json_block(response[0])
    output = output_parser.parse(final_response)

    return output

## 3. MAIN WORKFLOW

In [ ]:
def main(pdf_path):
    full_text = extract_text_from_pdf(pdf_path)
    sections = extract_sections(full_text)
    results = {}

    for key in ['abstract', 'introduction', 'methodology', 'results', 'conclusion']:
        print(f"\n\n===== {key.capitalize()} =====\n")
        section_text = sections.get(key, "")
        if section_text:
            # print(section_text[:500] + "..." if len(section_text) > 500 else section_text)
            try:
                simplified = Simplify_research_part(section_text)
                results[key] = simplified['explanation']
                # print(f"\nSimplified Explanation of {key.capitalize()}:\n{simplified}")
            except Exception as e:
                print(f"Could not simplify {key}: {str(e)}")
        else:
            print("Section not found.")

    return results

## Explain all research paper

In [ ]:
pdf_file_path = r"arxiv_pdfs\0704.1020.pdf"
answer = main(pdf_file_path)



===== Abstract =====

Could not simplify abstract: name 'model' is not defined


===== Introduction =====

Could not simplify introduction: name 'model' is not defined


===== Methodology =====

Could not simplify methodology: name 'model' is not defined


===== Results =====

Could not simplify results: name 'model' is not defined


===== Conclusion =====

Could not simplify conclusion: name 'model' is not defined


### Trying with GEMINI

In [3]:
%pip install -q google-generativeai

Note: you may need to restart the kernel to use updated packages.


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-intel 2.17.0 requires protobuf!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<5.0.0dev,>=3.20.3, but you have protobuf 5.29.4 which is incompatible.


In [5]:
%pip install python-dotenv

  Obtaining dependency information for python-dotenv from https://files.pythonhosted.org/packages/1e/18/98a99ad95133c6a6e2005fe89faedf294a748bd5dc803008059409ac9b1e/python_dotenv-1.1.0-py3-none-any.whl.metadata
Note: you may need to restart the kernel to use updated packages.


In [14]:
# import necessary libraries
import re
import PyPDF2
import google.generativeai as genai
from langchain.prompts import PromptTemplate
from langchain.output_parsers import StructuredOutputParser, ResponseSchema
from langchain.prompts import ChatPromptTemplate, HumanMessagePromptTemplate
from dotenv import load_dotenv
import os
import json

load_dotenv()  # Loads variables from .env into environment

# Get API key for Gemini
GEMINI_API_KEY = os.getenv("API_KEY")  # Make sure to set this in your .env file

# Configure Gemini
genai.configure(api_key=GEMINI_API_KEY)

# Initialize Gemini model
model = genai.GenerativeModel('gemini-1.5-flash-latest')

def generate_text(prompt, max_length=100, temperature=0.7):
    """Generate text using Gemini model"""
    try:
        # Configure generation parameters
        generation_config = genai.types.GenerationConfig(
            max_output_tokens=max_length * 4,  # Gemini uses tokens differently than words
            temperature=temperature,
        )
        
        response = model.generate_content(
            prompt,
            generation_config=generation_config
        )
        
        return [response.text]
    except Exception as e:
        print(f"Error generating text: {e}")
        return [f"Error: {str(e)}"]

# the section patterns to identify different parts of a research paper
SECTION_PATTERNS = {
    "abstract": r'\babstract\b',
    "introduction": r'\bintroduction\b',
    "methodology": r'\b(methodology|methods|materials and methods)\b',
    "results": r'\b(results|findings)\b',
    "conclusion": r'\b(conclusion|conclusions|discussion and conclusion)\b'
}

# Compile the regex patterns for section headers
SECTION_REGEX = {k: re.compile(v, re.IGNORECASE) for k, v in SECTION_PATTERNS.items()}

def extract_text_from_pdf(pdf_path):
    with open(pdf_path, 'rb') as file:
        reader = PyPDF2.PdfReader(file)
        text = ""
        for page in reader.pages:
            text += page.extract_text() + "\n"
        return text

def extract_sections(text):
    text = re.sub(r'\s+', ' ', text).strip()
    combined_pattern = r'|'.join(f'(?P<{k}>{v})' for k, v in SECTION_PATTERNS.items())
    section_header_regex = re.compile(combined_pattern, re.IGNORECASE)

    sections = {}
    matches = list(section_header_regex.finditer(text))

    for i, match in enumerate(matches):
        section_name = match.lastgroup
        start = match.start()
        end = matches[i + 1].start() if i + 1 < len(matches) else len(text)
        content = text[start:end]
        sections[section_name] = content.strip()

    return sections

def extract_json_block(text):
    """Extract JSON from markdown code blocks"""
    pattern = r'```json\s*(.*?)\s*```'
    matches = re.findall(pattern, text, re.DOTALL)
    
    if matches:
        return f"```json\n{matches[-1]}\n```"
    
    # If no markdown json found, try to find JSON directly
    try:
        # Look for JSON pattern in the text
        json_pattern = r'\{[^}]*"explanation"[^}]*\}'
        json_match = re.search(json_pattern, text, re.DOTALL)
        if json_match:
            return f"```json\n{json_match.group()}\n```"
    except:
        pass
    
    # If still no JSON found, wrap the entire response as explanation
    return f'```json\n{{"explanation": "{text.strip()}"}}\n```'

def Simplify_research_part(content):
    """Simplify research content using Gemini"""
    explanation_schema = ResponseSchema(
        name="explanation",
        description="A simplified and clear explanation of the provided research text."
    )
    output_parser = StructuredOutputParser.from_response_schemas([explanation_schema])
    format_instructions = output_parser.get_format_instructions()

    prompt_template = """\
    You are a helpful assistant that explains research papers in a simple way.

    TASK:
    - Read the provided research content carefully.
    - Return a clear and simplified explanation in one paragraph using plain, accurate language.
    - Avoid adding any information that is not in the original content.
    - The goal is to help anyone understand this part, even without a research background, while staying true to the original meaning.
    - Format your response as JSON with the key "explanation".

    Content:
    {content}

    Please provide your response in the following JSON format:
    {{
        "explanation": "Your simplified explanation here"
    }}

    {format_instructions}
    """
    
    prompt = PromptTemplate(
        template=prompt_template,
        input_variables=["content", "format_instructions"]
    )

    formatted_prompt = prompt.format(content=content, format_instructions=format_instructions)

    # Generate response with increased max_length for better explanations
    response = generate_text(formatted_prompt, max_length=500, temperature=0.7)
    
    # Extract JSON block
    final_response = extract_json_block(response[0])
    
    try:
        # Parse the output
        output = output_parser.parse(final_response)
        return output
    except Exception as e:
        # If parsing fails, return a basic structure
        print(f"Parsing error: {e}")
        # Try to extract explanation manually
        try:
            # Clean up the response and extract explanation
            cleaned_response = response[0].strip()
            if cleaned_response.startswith('"') and cleaned_response.endswith('"'):
                cleaned_response = cleaned_response[1:-1]
            return {"explanation": cleaned_response}
        except:
            return {"explanation": "Unable to simplify this section"}

def process_pdf(pdf_path):
    """
    Process the PDF content to extract text and summarize it using Gemini.
    """
    try:
        full_text = extract_text_from_pdf(pdf_path)
        sections = extract_sections(full_text)
        results = {}

        for key in ['abstract', 'introduction', 'methodology', 'results', 'conclusion']:
            print(f"\n\n===== {key.capitalize()} =====\n")
            section_text = sections.get(key, "")
            if section_text:
                # Limit section text length to avoid token limits
                if len(section_text) > 3000:
                    section_text = section_text[:3000] + "..."
                
                try:
                    simplified = Simplify_research_part(section_text)
                    results[key] = simplified['explanation']
                    print(f"✅ Successfully simplified {key}")
                except Exception as e:
                    print(f"❌ Could not simplify {key}: {str(e)}")
                    # Add error handling to continue processing other sections
                    results[key] = f"Error processing {key}: {str(e)}"
            else:
                print("Section not found.")
                results[key] = "Section not found in the document."

        return results
    
    except Exception as e:
        print(f"Error processing PDF: {str(e)}")
        return {"error": str(e)}


In [15]:

# Test function to verify Gemini is working
def test_gemini():
    """Test if Gemini is properly configured"""
    try:
        response = model.generate_content("Please respond with 'Gemini is working correctly!'")
        print(f"✅ Gemini test successful: {response.text}")
        return True
    except Exception as e:
        print(f"❌ Gemini test failed: {e}")
        return False

# Additional helper function for better error handling
def process_pdf_with_fallback(pdf_path):
    """
    Process PDF with fallback mechanisms
    """
    # First test if Gemini is working
    if not test_gemini():
        return {"error": "Gemini API is not properly configured"}
    
    return process_pdf(pdf_path)

In [19]:
pdf_file_path = r"arxiv_pdfs\0704.1020.pdf"
answer = process_pdf_with_fallback(pdf_file_path)

✅ Gemini test successful: Gemini is working correctly!



===== Abstract =====

✅ Successfully simplified abstract


===== Introduction =====

✅ Successfully simplified introduction


===== Methodology =====

✅ Successfully simplified methodology


===== Results =====

✅ Successfully simplified results


===== Conclusion =====

✅ Successfully simplified conclusion


In [20]:
# Print the final results   
print("\n\n===== Final Results =====\n")
for section, content in answer.items():
    print(f"{section}:\n{content}\n")



===== Final Results =====

abstract:
This research explores finding the shortest path in a graph where edge weights change unpredictably.  An algorithm is developed that, over many rounds, finds paths whose total weight is close to the best path possible in hindsight.  The algorithm's performance is proven to be efficient, scaling well with the number of rounds and edges.  The research also extends the algorithm to scenarios with limited information about edge weights and to situations where the best path itself changes over time.

introduction:
This research explores a decision-making process where someone repeatedly makes choices, each resulting in a loss depending on the environment's response.  The goal is to minimize total loss over time without making assumptions about how the losses are created.  The research focuses on performing as well as the best among a group of other decision-makers (experts), aiming to match their average loss in the long run.

methodology:
Existing met